<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/results/Mirabest_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import shutil
import subprocess

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!nvidia-smi

Fri Aug  5 14:06:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install astro-datasets --upgrade
!pip install tensorflow_datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for astro-datasets: filename=astro_datasets-0.0.9-py3-none-any.whl size=15986 sha256=97c300816d76fa38c75020805cd0e7babb8162c7b6df1450fa1cf19ae9039ec6
  Stored in directory: /root/.cache/pip/wheels/77/11/df/7eb516aa333b47883b39adabbe38508a09163be741b476dcda
Successfully built astro-datasets
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
shutil.rmtree('bnn_hmc', ignore_errors=True)

In [ ]:
!git clone https://github.com/adammoss/bnn_hmc

Cloning into 'bnn_hmc'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 149 (delta 94), reused 111 (delta 56), pack-reused 0
Receiving objects: 100% (149/149), 467.83 KiB | 717.00 KiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
# https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

!pip install --upgrade https://storage.googleapis.com/jax-releases/cuda111/jaxlib-0.1.65+cuda111-cp37-none-manylinux2010_x86_64.whl
!pip install jax==0.2.12

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 189.4 MB 22 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.3.14+cuda11.cudnn805
    Uninstalling jaxlib-0.3.14+cuda11.cudnn805:
      Successfully uninstalled jaxlib-0.3.14+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 590 kB 29.1 MB/s 
  Created wheel for jax: filename=jax-0.2.12-py3-none-any.whl size=682487 sha256=08d25b66b1f82a74744758c8a9262ce9817d1c641a4800c847ad073e0cf3df4f
  Stored in directory: /root/.cache/pip/wheels/f6/4d/e5/73eec5070b77f25664c67bd793d4eb97f41bbf9be7afafd15e
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.3.14
    Uninstalling jax-0.3.14:
      Successfully uninstalled jax-0.3.14


In [ ]:
!pip install dm-haiku==0.0.5.dev0 optax==0.0.6 chex==0.0.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 284 kB 28.9 MB/s 
     |████████████████████████████████| 96 kB 3.7 MB/s 
     |████████████████████████████████| 51 kB 402 kB/s 


In [ ]:
!pip freeze

absl-py==1.2.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astro-datasets==0.0.9
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==2.1.9
autograd==1.4
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.6.15
cffi==1.15.1
cftime==1.6.1
chardet==3.0.4
charset-normalizer==2.1.0
chex==0.0.6
click==7.1.2
clikit==0.6.2
cloudpickle==1.3.0
cmake==3.22.6
cmdstanpy==1.0.4
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dask==2.12.0
datasci

SGD

Mirabest

In [ ]:
!python3 bnn_hmc/scripts/run_sgd.py --seed=1 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/ \
  --dataset_name=mirabest/confident --model_name=lenet \
  --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=80 --patience 10

JAX sees the following devices: [GpuDevice(id=0, process_index=0)]
TF sees the following devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
2022-08-05 08:51:19.232490: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0% 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0% 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100% 1/1 [00:01<00:00,  1.41s/ url]
Dl Size...: 0 MiB [00:01, ? MiB/s]

Dl Completed...: 100% 1/1 [00:01<00:00,  1.41s/ url]
Dl Size...: 0 MiB [00:01, ? MiB/s]

Extraction completed...:   0% 0/1 [00

In [ ]:
!python3 bnn_hmc/scripts/run_sgd.py --seed=1 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/ \
  --dataset_name=mirabest/confident --model_name=lenet \
  --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=80 --eval_split test

JAX sees the following devices: [GpuDevice(id=0, process_index=0)]
TF sees the following devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
tcmalloc: large alloc 1183137792 bytes == 0x115da000 @  0x7f4e6f489887 0x7f4e5d58e7ce 0x7f4e5d5932c2 0x7f4e5d5998a2 0x7f4e5d599e7d 0x7f4e5d59a1e9 0x7f4e5cb8bf11 0x7f4e5cb8719f 0x7f4e5d0a2d01 0x7f4e5cba6594 0x7f4e5cba7126 0x7f4e5c2bca69 0x7f4e5c2b64e0 0x7f4e5c2c0a3c 0x7f4e5c2b0ba6 0x7f4e5c07dfa5 0x7f4e5c073961 0x593784 0x594731 0x548cc1 0x51566f 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x598ef4 0x500d94 0x5a7284 0x5a9cf4
Continuing the run from the last saved checkpoint
bnn_hmc/scripts/run_sgd.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predict_fn(net_apply, params, net_state, test_se

In [ ]:
!python3 bnn_hmc/scripts/run_sgd.py --seed=2 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/ \
  --dataset_name=mirabest/confident --model_name=lenet \
  --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=80 --patience 10

JAX sees the following devices: [GpuDevice(id=0, process_index=0)]
TF sees the following devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
tcmalloc: large alloc 1183137792 bytes == 0x10c14000 @  0x7f9d9f199887 0x7f9d8d29e7ce 0x7f9d8d2a32c2 0x7f9d8d2a98a2 0x7f9d8d2a9e7d 0x7f9d8d2aa1e9 0x7f9d8c89bf11 0x7f9d8c89719f 0x7f9d8cdb2d01 0x7f9d8c8b6594 0x7f9d8c8b7126 0x7f9d8bfcca69 0x7f9d8bfc64e0 0x7f9d8bfd0a3c 0x7f9d8bfc0ba6 0x7f9d8bd8dfa5 0x7f9d8bd83961 0x593784 0x594731 0x548cc1 0x51566f 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x598ef4 0x500d94 0x5a7284 0x5a9cf4
Starting from random initialization with provided seed
/content/bnn_hmc/bnn_hmc/utils/script_utils.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predict_fn(net_apply, p

In [ ]:
!python3 bnn_hmc/scripts/run_sgd.py --seed=2 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/ \
  --dataset_name=mirabest/confident --model_name=lenet \
  --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=80 --eval_split test

JAX sees the following devices: [GpuDevice(id=0, process_index=0)]
TF sees the following devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
tcmalloc: large alloc 1183137792 bytes == 0x106ee000 @  0x7f159df71887 0x7f158c0767ce 0x7f158c07b2c2 0x7f158c0818a2 0x7f158c081e7d 0x7f158c0821e9 0x7f158b673f11 0x7f158b66f19f 0x7f158bb8ad01 0x7f158b68e594 0x7f158b68f126 0x7f158ada4a69 0x7f158ad9e4e0 0x7f158ada8a3c 0x7f158ad98ba6 0x7f158ab65fa5 0x7f158ab5b961 0x593784 0x594731 0x548cc1 0x51566f 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x598ef4 0x500d94 0x5a7284 0x5a9cf4
Continuing the run from the last saved checkpoint
bnn_hmc/scripts/run_sgd.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  predict_fn(net_apply, params, net_state, test_se

In [ ]:
for i in range(0,10):
 base_cmd = ['python3', 'bnn_hmc/scripts/run_sgd.py', '--seed=%s' % i, '--weight_decay=10', '--dir=runs/sgd/mirabestc/lenet/%s/' % i, 
 '--dataset_name=mirabest/confident', '--model_name=lenet', '--init_step_size=3e-7', '--num_epochs=200', 
 '--eval_freq=5', '--batch_size=53', '--patience=10']
 train_cmd = base_cmd + ['--train_split=train[:80%]', '--test_split=train[80%:]']
 eval_cmd = base_cmd + ['--eval_split=test']
 print(' '.join(train_cmd))
 subprocess.run(train_cmd, capture_output=True)
 subprocess.run(eval_cmd, capture_output=True)

python3 bnn_hmc/scripts/run_sgd.py --seed=0 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/0/ --dataset_name=mirabest/confident --model_name=lenet --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=53 --patience=10 --train_split=train[:80%] --test_split=train[80%:]
python3 bnn_hmc/scripts/run_sgd.py --seed=1 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/1/ --dataset_name=mirabest/confident --model_name=lenet --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=53 --patience=10 --train_split=train[:80%] --test_split=train[80%:]
python3 bnn_hmc/scripts/run_sgd.py --seed=2 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/2/ --dataset_name=mirabest/confident --model_name=lenet --init_step_size=3e-7 --num_epochs=200 --eval_freq=5 --batch_size=53 --patience=10 --train_split=train[:80%] --test_split=train[80%:]
python3 bnn_hmc/scripts/run_sgd.py --seed=3 --weight_decay=10 --dir=runs/sgd/mirabestc/lenet/3/ --dataset_name=mirabest/confident --model_name=lenet --

In [ ]:
import numpy as np
#prediction_path = '/content/runs/sgd/mirabestc/lenet/sgd_mom_0.9__lr_sch_i_3e-07___epochs_200_wd_10.0_batchsize_80_temp_1.0__seed_1/predictions.npy'
#test = np.load(prediction_path)
#print(test)

data = np.load('/content/runs/sgd/mirabestc/lenet/0/sgd_mom_0.9__lr_sch_i_3e-07___epochs_200_wd_10.0_batchsize_53_temp_1.0__seed_0/test_set.npy')
#print(np.argmax(test, axis=-1) == data)
#print(np.size(data))

In [ ]:

for a in range(0, 10):
  prediction = np.load('/content/runs/sgd/mirabestc/lenet/' + str(a) + '/sgd_mom_0.9__lr_sch_i_3e-07___epochs_200_wd_10.0_batchsize_53_temp_1.0__seed_' 
                       + str(a) + '/predictions.npy')
  print('Seed = ' + str(a))
  #for _ in range(10):
  correct, n = 0, 0
  
  correct += np.sum(np.argmax(prediction, axis=-1) == data)
  n += np.size(data)
  print('Accuracy:',correct / n)



Seed = 0
Accuracy: 0.8942307692307693
Seed = 1
Accuracy: 0.8942307692307693
Seed = 2
Accuracy: 0.8846153846153846
Seed = 3
Accuracy: 0.8942307692307693
Seed = 4
Accuracy: 0.9038461538461539
Seed = 5
Accuracy: 0.9038461538461539
Seed = 6
Accuracy: 0.8653846153846154
Seed = 7
Accuracy: 0.9038461538461539
Seed = 8
Accuracy: 0.8942307692307693
Seed = 9
Accuracy: 0.8653846153846154


In [ ]:
samples_iter = 10
softmax = []



for a in range(0, 10):
  prediction = np.load('/content/runs/sgd/mirabestc/lenet/' + str(a) + '/sgd_mom_0.9__lr_sch_i_3e-07___epochs_200_wd_10.0_batchsize_53_temp_1.0__seed_' 
                       + str(a) + '/predictions.npy')
  
  softmax.append(prediction)

softmax = np.array(softmax)



def entropy_MI(softmax, samples_iter):

    class_0 = softmax[:][0]
    class_1 = softmax[:][1]
    #biased estimator of predictive entropy, bias will reduce as samples_iter is increased
    predictive_entropy = -((np.sum(class_0)/samples_iter) * np.log(np.sum(class_0)/samples_iter) + (np.sum(class_1)/samples_iter) * np.log(np.sum(class_1)/samples_iter))
    
    #Orignal: -((np.sum(class_0)/samples_iter) * np.log(np.sum(class_0)/samples_iter) + (np.sum(class_1)/samples_iter) * np.log(np.sum(class_1)/samples_iter))
    

    mutual_info = predictive_entropy + np.sum((class_0 * np.log(class_0)) + (class_1 * np.log(class_1)))/samples_iter

    #Original: predictive_entropy + np.sum(class_0*np.log(class_0))/samples_iter +  np.sum(class_1*np.log(class_1))/samples_iter
    
    #entropy of a single pass
    class0 = softmax[:][0][0]
    class1 = softmax[:][1][0]
    entropy_singlepass = -np.sum((class0 * np.log(class0)) + (class1 * np.log(class1)))/samples_iter

    #Original: -(class0 * np.log(class0) + class1 * np.log(class1))

    #print("Entropy of a single pass:", entropy_singlepass)
    return predictive_entropy, mutual_info, entropy_singlepass



predictive_entropy, mutual_info, entropy_singlepass = entropy_MI(softmax, samples_iter)


print("Predictive Entropy:", predictive_entropy)
print("Mutual Information:", mutual_info)
print("Entropy of a single pass:", entropy_singlepass)

Predictive Entropy: -48.7095607678644
Mutual Information: -54.417037955974756
Entropy of a single pass: 5.707477188110351
